In [1]:
import pandas as pd
import numpy as np

data = "/mnt/d/Data/mangaki-data-challenge/"

record = pd.read_csv(data+'watched.csv', dtype={
    'user_id': np.int16,
    'work_id': np.int16,
    'rating': 'category'
})

train_full = pd.read_csv(data+'train_withcv.csv', dtype={
    'user_id': np.int16,
    'work_id': np.int16,
    'rating': np.int8,
    'cv': np.int8
})

test = pd.read_csv(data+'test.csv', dtype={
    'user_id': np.int16,
    'work_id': np.int16
})

title = pd.read_csv(data+'titles.csv', dtype={
    'work_id': np.int16,
    'title': str,
    'category': str
})
catlut = {'anime': 0, 'manga': 1, 'album':2}
ratelut = {'dislike':1.0, 'like':3.0, 'love':4.0, 'neutral':2.0}
title = title[['work_id', 'category']].set_index('work_id').applymap(lambda x: catlut[x])
record = record.merge(title, left_on='work_id', right_index=True)
record['score']=record.rating.map(lambda x: ratelut[x]).astype(float)

In [2]:
u1 = pd.crosstab(record[record.category==0].user_id, record[record.category==0].rating).add_prefix('user_anime_').apply(lambda r: r/r.sum(), axis=1)
u2 = pd.crosstab(record[record.category==1].user_id, record[record.category==1].rating).add_prefix('user_manga_').apply(lambda r: r/r.sum(), axis=1)

In [3]:
u3 = record[record.category==0][['user_id', 'score']].groupby(by='user_id')['score'].agg(['mean', 'std']).rename(columns={'mean':'user_anime_mean', 'std':'user_anime_std'})
u4 = record[record.category==1][['user_id', 'score']].groupby(by='user_id')['score'].agg(['mean', 'std']).rename(columns={'mean':'user_manga_mean', 'std':'user_manga_std'})

In [4]:
i1 = pd.crosstab(record.work_id, record.rating).add_prefix('work_').apply(lambda r: r/r.sum(), axis=1)
i2 = record[['work_id', 'score']].groupby(by='work_id')['score'].agg(['mean', 'std']).add_prefix('item_')

In [5]:
for cv in [1,2,3]:
    valid = train_full[train_full.cv==cv]
    train = train_full[~train_full.cv.isin([cv])]
    train = train.merge(title.add_prefix('item_'), left_on='work_id', right_index=True, how='left').\
                  merge(u1, left_on='user_id', right_index=True, how='left').\
                  merge(u2, left_on='user_id', right_index=True, how='left').\
                  merge(u3, left_on='user_id', right_index=True, how='left').\
                  merge(u4, left_on='user_id', right_index=True, how='left').\
                  merge(i1, left_on='work_id', right_index=True, how='left').\
                  merge(i2, left_on='work_id', right_index=True, how='left')
    valid = valid.merge(title.add_prefix('item_'), left_on='work_id', right_index=True, how='left').\
                  merge(u1, left_on='user_id', right_index=True, how='left').\
                  merge(u2, left_on='user_id', right_index=True, how='left').\
                  merge(u3, left_on='user_id', right_index=True, how='left').\
                  merge(u4, left_on='user_id', right_index=True, how='left').\
                  merge(i1, left_on='work_id', right_index=True, how='left').\
                  merge(i2, left_on='work_id', right_index=True, how='left')
    train = train.drop(['cv'], axis=1)
    train.to_csv(data+"baseline/train_{0}.csv".format(cv), index=False)
    valid = valid.drop(['cv'], axis=1)
    valid.to_csv(data+"baseline/valid_{0}.csv".format(cv), index=False)

In [6]:
train = train_full
train = train.merge(title.add_prefix('item_'), left_on='work_id', right_index=True, how='left').\
                  merge(u1, left_on='user_id', right_index=True, how='left').\
                  merge(u2, left_on='user_id', right_index=True, how='left').\
                  merge(u3, left_on='user_id', right_index=True, how='left').\
                  merge(u4, left_on='user_id', right_index=True, how='left').\
                  merge(i1, left_on='work_id', right_index=True, how='left').\
                  merge(i2, left_on='work_id', right_index=True, how='left')
                  
train = train.drop(['cv'], axis=1)
train.to_csv(data+"baseline/train_0.csv", index=False)

test = test.merge(title.add_prefix('item_'), left_on='work_id', right_index=True, how='left').\
                  merge(u1, left_on='user_id', right_index=True, how='left').\
                  merge(u2, left_on='user_id', right_index=True, how='left').\
                  merge(u3, left_on='user_id', right_index=True, how='left').\
                  merge(u4, left_on='user_id', right_index=True, how='left').\
                  merge(i1, left_on='work_id', right_index=True, how='left').\
                  merge(i2, left_on='work_id', right_index=True, how='left')
                  
test.to_csv(data+"baseline/test_0.csv", index=False)

W2V feature

In [15]:
w2vdim=50

In [23]:
from gensim.models.word2vec import Word2Vec
from random import sample

record_positive = record[record.rating.isin(['like', 'love'])]
record_positive_effectiveuser = record_positive[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
record_positive_effectiveuser = record_positive_effectiveuser[record_positive_effectiveuser.work_count>5]
record_positive = record_positive[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})
record_positive = pd.concat((record_positive, record_positive_effectiveuser), axis=1, join='inner')

sentences = []
for itm in record_positive.iterrows():
    t = ['i{0}'.format(str(i)) for i in itm[1].work_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_positive = Word2Vec(sentences, size=w2vdim)

item_w2v_positive = []
item_index = []
for item in range(9897):
    if 'i{0}'.format(item) in w2v_positive:
        item_index.append(item)
        item_w2v_positive.append(w2v_positive['i{0}'.format(item)])
item_w2v_positive = np.vstack(item_w2v_positive)
i4 = pd.DataFrame(data = item_w2v_positive, index=item_index, columns=['itemw2vpos_{0}'.format(i) for i in range(w2vdim)])
i4.index.name='work_id'

i4.reset_index().to_csv(data+"features/item_w2v_shuffled_{0}d.csv".format(w2vdim), index=False)

In [24]:
# for user
record_positive = record[record.rating.isin(['like', 'love'])]
record_positive_effectiveitem = record_positive[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
record_positive_effectiveitem = record_positive_effectiveitem[record_positive_effectiveitem.user_count>5]
record_positive = record_positive[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})
record_positive = pd.concat((record_positive, record_positive_effectiveitem), axis=1, join='inner')

sentences = []
for itm in record_positive.iterrows():
    t = ['u{0}'.format(str(i)) for i in itm[1].user_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_positive = Word2Vec(sentences, size=w2vdim)

user_w2v_positive = []
user_index = []
for user in range(1983):
    if 'u{0}'.format(user) in w2v_positive:
        user_index.append(user)
        user_w2v_positive.append(w2v_positive['u{0}'.format(user)])
user_w2v_positive = np.vstack(user_w2v_positive)
u4 = pd.DataFrame(data = user_w2v_positive, index=user_index, columns=['userw2vpos_{0}'.format(i) for i in range(w2vdim)])
u4.index.name='user_id'

u4.reset_index().to_csv(data+"features/user_w2v_shuffled_{0}d.csv".format(w2vdim), index=False)

In [16]:
from gensim.models.word2vec import Word2Vec
from random import sample

record_negative = record[record.rating.isin(['dislike'])]
record_negative_effectiveuser = record_negative[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
record_negative_effectiveuser = record_negative_effectiveuser[record_negative_effectiveuser.work_count>5]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})
record_negative = pd.concat((record_negative, record_negative_effectiveuser), axis=1, join='inner')

sentences = []
for itm in record_negative.iterrows():
    t = ['i{0}'.format(str(i)) for i in itm[1].work_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_negative = Word2Vec(sentences, size=w2vdim)

item_w2v_negative = []
item_index = []
for item in range(9897):
    if 'i{0}'.format(item) in w2v_negative:
        item_index.append(item)
        item_w2v_negative.append(w2v_negative['i{0}'.format(item)])
item_w2v_negative = np.vstack(item_w2v_negative)
i4 = pd.DataFrame(data = item_w2v_negative, index=item_index, columns=['itemw2vneg_{0}'.format(i) for i in range(w2vdim)])
i4.index.name='work_id'

i4.reset_index().to_csv(data+"features/item_w2vneg_shuffled_{0}d.csv".format(w2vdim), index=False)

In [17]:
record_negative = record[record.rating.isin(['dislike'])]
record_negative_effectiveitem = record_negative[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
record_negative_effectiveitem = record_negative_effectiveitem[record_negative_effectiveitem.user_count>5]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})
record_negative = pd.concat((record_negative, record_negative_effectiveitem), axis=1, join='inner')

sentences = []
for itm in record_negative.iterrows():
    t = ['u{0}'.format(str(i)) for i in itm[1].user_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_negative = Word2Vec(sentences, size=w2vdim)

user_w2v_negative = []
user_index = []
for user in range(1983):
    if 'u{0}'.format(user) in w2v_negative:
        user_index.append(user)
        user_w2v_negative.append(w2v_negative['u{0}'.format(user)])
user_w2v_negative = np.vstack(user_w2v_negative)
u4 = pd.DataFrame(data = user_w2v_negative, index=user_index, columns=['userw2vneg_{0}'.format(i) for i in range(w2vdim)])
u4.index.name='user_id'

u4.reset_index().to_csv(data+"features/user_w2vneg_shuffled_{0}d.csv".format(w2vdim), index=False)

LDA feature

In [25]:
num_topics = 10

In [26]:
from gensim.models.ldamodel import LdaModel

record_positive = record[record.rating.isin(['like', 'love', 'neutral'])]
record_positive_effectiveuser = record_positive[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
record_positive_effectiveuser = record_positive_effectiveuser[record_positive_effectiveuser.work_count>5]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='user_id').agg(lambda x: list(x))
record_positive = pd.concat((record_positive, record_positive_effectiveuser), axis=1, join='inner')

corpus = []
dictionary = {}
cnt=0
ratlut = {'love': 1.5, 'like': 1, 'neutral': 0.5}
for itm in record_positive.iterrows():
    for work in itm[1].work_id:
        strwork = 'i{0}'.format(work)
        if strwork not in dictionary:
            dictionary[strwork]=cnt
            cnt+=1
    corpus.append([(dictionary['i{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].work_id, itm[1].rating)])
invdict = dict(zip(dictionary.values(), dictionary.keys()))

lda_user = LdaModel(corpus=corpus, num_topics=num_topics, id2word=invdict)

record_positive = record[record.rating.isin(['like', 'love', 'neutral'])]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='user_id').agg(lambda x: list(x))

corpus = []
for itm in record_positive.iterrows():
    corpus.append([(dictionary['i{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].work_id, itm[1].rating) if 'i{0}'.format(i) in dictionary])

user_lda_positive = []
for i, topic in enumerate(lda_user[corpus]):
    temp = np.zeros(num_topics)
    for itm in topic:
        temp[itm[0]] = itm[1]
    user_lda_positive.append(temp)
user_lda_positive = np.vstack(user_lda_positive)
u5 = pd.DataFrame(data = user_lda_positive, index=record_positive.index, columns=['user_ldapos_{0}'.format(i) for i in range(num_topics)])

u5.reset_index().to_csv(data+"features/user_lda_{0}d.csv".format(num_topics), index=False)

In [27]:
record_positive = record[record.rating.isin(['like', 'love', 'neutral'])]
record_positive_effectiveitem = record_positive[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
record_positive_effectiveitem = record_positive_effectiveitem[record_positive_effectiveitem.user_count>5]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='work_id').agg(lambda x: list(x))
record_positive = pd.concat((record_positive, record_positive_effectiveitem), axis=1, join='inner')

corpus = []
dictionary = {}
cnt=0
ratlut = {'love': 1.5, 'like': 1, 'neutral': 0.5}
for itm in record_positive.iterrows():
    for user in itm[1].user_id:
        struser = 'u{0}'.format(user)
        if struser not in dictionary:
            dictionary[struser]=cnt
            cnt+=1
    corpus.append([(dictionary['u{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].user_id, itm[1].rating)])
invdict = dict(zip(dictionary.values(), dictionary.keys()))

lda_item = LdaModel(corpus=corpus, num_topics=num_topics, id2word=invdict)

record_positive = record[record.rating.isin(['like', 'love', 'neutral'])]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='work_id').agg(lambda x: list(x))

corpus = []
for itm in record_positive.iterrows():
    corpus.append([(dictionary['u{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].user_id, itm[1].rating) if 'u{0}'.format(i) in dictionary])

item_lda_positive = []
for i, topic in enumerate(lda_item[corpus]):
    temp = np.zeros(num_topics)
    for itm in topic:
        temp[itm[0]] = itm[1]
    item_lda_positive.append(temp)
item_lda_positive = np.vstack(item_lda_positive)
i5 = pd.DataFrame(data = item_lda_positive, index=record_positive.index, columns=['item_ldapos_{0}'.format(i) for i in range(num_topics)])

i5.reset_index().to_csv(data+"features/item_lda_{0}d.csv".format(num_topics), index=False)

In [31]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora

record_negative = record[record.rating.isin(['dislike'])]
record_negative_effectiveuser = record_negative[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
record_negative_effectiveuser = record_negative_effectiveuser[record_negative_effectiveuser.work_count>5]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})
record_negative = pd.concat((record_negative, record_negative_effectiveuser), axis=1, join='inner')

sentences = []
for itm in record_negative.iterrows():
    sentences.append(['i{0}'.format(str(i)) for i in itm[1].work_list])
    
dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(rc) for rc in sentences]

lda_user = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)

record_negative = record[record.rating.isin(['dislike'])]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})

sentences = []
for itm in record_negative.iterrows():
    sentences.append(['i{0}'.format(str(i)) for i in itm[1].work_list])
corpus = [dictionary.doc2bow(rc) for rc in sentences]

user_lda_negative = []
for i, topic in enumerate(lda_user[corpus]):
    temp = np.zeros(num_topics)
    for itm in topic:
        temp[itm[0]] = itm[1]
    user_lda_negative.append(temp)
user_lda_negative = np.vstack(user_lda_negative)
u5 = pd.DataFrame(data = user_lda_negative, index=record_negative.index, columns=['user_ldapos_{0}'.format(i) for i in range(num_topics)])

u5.reset_index().to_csv(data+"features/user_ldaneg_{0}d.csv".format(num_topics), index=False)

In [32]:
record_negative = record[record.rating.isin(['dislike'])]
record_negative_effectiveitem = record_negative[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
record_negative_effectiveitem = record_negative_effectiveitem[record_negative_effectiveitem.user_count>5]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})
record_negative = pd.concat((record_negative, record_negative_effectiveitem), axis=1, join='inner')

sentences = []
for itm in record_negative.iterrows():
    sentences.append(['u{0}'.format(str(i)) for i in itm[1].user_list])
    
dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(rc) for rc in sentences]

lda_item = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)

record_negative = record[record.rating.isin(['like', 'love'])]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})

sentences = []
for itm in record_negative.iterrows():
    sentences.append(['u{0}'.format(str(i)) for i in itm[1].user_list])
corpus = [dictionary.doc2bow(rc) for rc in sentences]

item_lda_negative = []
for i, topic in enumerate(lda_item[corpus]):
    temp = np.zeros(num_topics)
    for itm in topic:
        temp[itm[0]] = itm[1]
    item_lda_negative.append(temp)
item_lda_negative = np.vstack(item_lda_negative)
i5 = pd.DataFrame(data = item_lda_negative, index=record_negative.index, columns=['item_ldapos_{0}'.format(i) for i in range(num_topics)])

i5.reset_index().to_csv(data+"features/item_ldaneg_{0}d.csv".format(num_topics), index=False)

convert to libfm format

In [11]:
from scipy.sparse import coo_matrix, hstack
from sklearn.datasets import dump_svmlight_file


path = "/mnt/d/Data/mangaki-data-challenge/"

train = pd.read_csv(path+"latest/train_0.csv")
valid = pd.read_csv(path+"latest/test_0.csv")

data = []
row = []
col = []
for i, itm in enumerate(train.iterrows()):
    row.append(i)
    col.append(itm[1].user_id)
    data.append(1.0)
    row.append(i)
    col.append(itm[1].work_id+1983)
    data.append(1.0)
    row.append(i)
    col.append(itm[1].item_category+1983+9897)
    data.append(1.0)
    
basic = coo_matrix((data, (row, col)), shape=(train.shape[0], 1983+9897+3))
ext = train.ix[:, 4:].fillna(0).values

In [13]:
ext_prob = np.hstack([ext[:, 0:8], ext[:, 12:16], ext[:, 102:142]])

In [14]:
ext_nonprob = np.hstack([ext[:, 8:12], ext[:, 16:102], ext[:, 142:]])

In [11]:
from sklearn.preprocessing import MaxAbsScaler

normalizer = MaxAbsScaler()
ext = normalizer.fit_transform(ext_nonprob)

value = hstack([basic, ext])
dump_svmlight_file(value, train.rating.values, path+"libfm/train_0.csv")

In [18]:
from scipy.sparse import coo_matrix, hstack
from sklearn.datasets import dump_svmlight_file
from sklearn.preprocessing import MaxAbsScaler


path = "/mnt/d/Data/mangaki-data-challenge/"
for cv in [1,2,3]:
    train = pd.read_csv(path+"latest/train_{0}.csv".format(cv))
    valid = pd.read_csv(path+"latest/valid_{0}.csv".format(cv))
    
    data = []
    row = []
    col = []
    for i, itm in enumerate(train.iterrows()):
        row.append(i)
        col.append(itm[1].user_id)
        data.append(1.0)
        row.append(i)
        col.append(itm[1].work_id+1983)
        data.append(1.0)
        row.append(i)
        col.append(itm[1].item_category+1983+9897)
        data.append(1.0)

    basic = coo_matrix((data, (row, col)), shape=(train.shape[0], 1983+9897+3))
    ext = train.ix[:, 4:].fillna(0).values
    ext_prob = np.hstack([ext[:, 0:8], ext[:, 12:16], ext[:, 102:142]])
    ext_nonprob = np.hstack([ext[:, 8:12], ext[:, 16:102], ext[:, 142:]])
    normalizer = MaxAbsScaler()
    ext_nonprob = normalizer.fit_transform(ext_nonprob)
    value = hstack([basic, ext_prob, ext_nonprob])
    dump_svmlight_file(value, train.rating.values, path+"libfm/train_{0}.csv".format(cv))
    
    data = []
    row = []
    col = []
    for i, itm in enumerate(valid.iterrows()):
        row.append(i)
        col.append(itm[1].user_id)
        data.append(1.0)
        row.append(i)
        col.append(itm[1].work_id+1983)
        data.append(1.0)
        row.append(i)
        col.append(itm[1].item_category+1983+9897)
        data.append(1.0)

    basic = coo_matrix((data, (row, col)), shape=(valid.shape[0], 1983+9897+3))
    ext = valid.ix[:, 4:].fillna(0).values
    ext_prob = np.hstack([ext[:, 0:8], ext[:, 12:16], ext[:, 102:142]])
    ext_nonprob = np.hstack([ext[:, 8:12], ext[:, 16:102], ext[:, 142:]])
    ext_nonprob = normalizer.transform(ext_nonprob)
    value = hstack([basic, ext_prob, ext_nonprob])
    dump_svmlight_file(value, valid.rating.values, path+"libfm/valid_{0}.csv".format(cv))

In [19]:
from scipy.sparse import coo_matrix, hstack
from sklearn.datasets import dump_svmlight_file


path = "/mnt/d/Data/mangaki-data-challenge/"

train = pd.read_csv(path+"latest/train_0.csv")
valid = pd.read_csv(path+"latest/test_0.csv")

data = []
row = []
col = []
for i, itm in enumerate(train.iterrows()):
    row.append(i)
    col.append(itm[1].user_id)
    data.append(1.0)
    row.append(i)
    col.append(itm[1].work_id+1983)
    data.append(1.0)
    row.append(i)
    col.append(itm[1].item_category+1983+9897)
    data.append(1.0)

basic = coo_matrix((data, (row, col)), shape=(train.shape[0], 1983+9897+3))
ext = train.ix[:, 4:].fillna(0).values
ext_prob = np.hstack([ext[:, 0:8], ext[:, 12:16], ext[:, 102:142]])
ext_nonprob = np.hstack([ext[:, 8:12], ext[:, 16:102], ext[:, 142:]])
normalizer = MaxAbsScaler()
ext_nonprob = normalizer.fit_transform(ext_nonprob)
value = hstack([basic, ext_prob, ext_nonprob])
dump_svmlight_file(value, train.rating.values, path+"libfm/train_0.csv")

data = []
row = []
col = []
for i, itm in enumerate(valid.iterrows()):
    row.append(i)
    col.append(itm[1].user_id)
    data.append(1.0)
    row.append(i)
    col.append(itm[1].work_id+1983)
    data.append(1.0)
    row.append(i)
    col.append(itm[1].item_category+1983+9897)
    data.append(1.0)

basic = coo_matrix((data, (row, col)), shape=(valid.shape[0], 1983+9897+3))
ext = valid.ix[:, 3:].fillna(0).values
ext_prob = np.hstack([ext[:, 0:8], ext[:, 12:16], ext[:, 102:142]])
ext_nonprob = np.hstack([ext[:, 8:12], ext[:, 16:102], ext[:, 142:]])
ext_nonprob = normalizer.transform(ext_nonprob)
value = hstack([basic, ext_prob, ext_nonprob])
dump_svmlight_file(value, np.zeros(valid.shape[0]), path+"libfm/test_0.csv")

In [20]:
for i, item in enumerate(train.columns):
    print("{0}\t{1}".format(i, item))

0	user_id
1	work_id
2	rating
3	item_category
4	user_anime_dislike
5	user_anime_like
6	user_anime_love
7	user_anime_neutral
8	user_manga_dislike
9	user_manga_like
10	user_manga_love
11	user_manga_neutral
12	user_anime_mean
13	user_anime_std
14	user_manga_mean
15	user_manga_std
16	work_dislike
17	work_like
18	work_love
19	work_neutral
20	item_mean
21	item_std
22	itemw2vpos_0_x
23	itemw2vpos_1_x
24	itemw2vpos_2_x
25	itemw2vpos_3_x
26	itemw2vpos_4_x
27	itemw2vpos_5_x
28	itemw2vpos_6_x
29	itemw2vpos_7_x
30	itemw2vpos_8_x
31	itemw2vpos_9_x
32	userw2vpos_0_x
33	userw2vpos_1_x
34	userw2vpos_2_x
35	userw2vpos_3_x
36	userw2vpos_4_x
37	userw2vpos_5_x
38	userw2vpos_6_x
39	userw2vpos_7_x
40	userw2vpos_8_x
41	userw2vpos_9_x
42	itemw2vpos_0_y
43	itemw2vpos_1_y
44	itemw2vpos_2_y
45	itemw2vpos_3_y
46	itemw2vpos_4_y
47	itemw2vpos_5_y
48	itemw2vpos_6_y
49	itemw2vpos_7_y
50	itemw2vpos_8_y
51	itemw2vpos_9_y
52	itemw2vpos_10
53	itemw2vpos_11
54	itemw2vpos_12
55	itemw2vpos_13
56	itemw2vpos_14
57	itemw2vpos_1

In [26]:
value

<100015x12056 sparse matrix of type '<class 'numpy.float64'>'
	with 13568306 stored elements in COOrdinate format>

In [20]:
valid.head()

,user_id,work_id,user_dislike,user_like,user_love,user_neutral,user_mean,user_std,work_dislike,work_like,...,user_lsi_10,user_lsi_11,user_lsi_12,user_lsi_13,user_lsi_14,user_lsi_15,user_lsi_16,user_lsi_17,user_lsi_18,user_lsi_19
0,486,1086,0.119565,0.728261,0.0,0.152174,2.608696,0.694666,0.279412,0.225490,...,-0.050692,-0.449053,0.127121,-0.140252,0.096404,0.083358,0.395873,0.360985,0.810576,-0.536926
1,1509,3296,0.048276,0.841379,0.0,0.110345,2.793103,0.512301,0.121212,0.686869,...,0.718754,-1.042719,-0.826651,-0.360414,0.482218,1.215442,-0.891306,-0.507401,0.326895,-0.237996
2,617,1086,0.027778,0.833333,0.0,0.138889,2.805556,0.467177,0.279412,0.225490,...,0.621460,-0.126613,0.096844,0.145862,0.655418,-1.152701,-0.642227,-0.115588,0.009927,-0.146983
3,270,9648,0.152597,0.629870,0.0,0.217532,2.477273,0.745979,0.062500,0.666667,...,0.535124,0.283546,-0.197112,0.362576,-0.067089,-2.450179,-0.811268,-1.291036,0.005630,0.588306
4,459,3647,0.031373,0.450980,0.0,0.517647,2.419608,0.554516,0.134146,0.707317,...,0.272404,-0.392137,0.318417,-0.458198,-0.334445,-0.021697,0.386012,-0.147479,1.022211,-0.370925


Using lsi

In [21]:
num_topic=20

In [17]:
record_positive = record
record_positive_effectiveuser = record_positive[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
record_positive_effectiveuser = record_positive_effectiveuser[record_positive_effectiveuser.work_count>5]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='user_id').agg(lambda x: list(x))
record_positive = pd.concat((record_positive, record_positive_effectiveuser), axis=1, join='inner')

corpus = []
dictionary = {}
cnt=0
ratlut = {'love': 1.5, 'like': 1, 'neutral': 0.5, 'dislike':-1}
for itm in record_positive.iterrows():
    for work in itm[1].work_id:
        strwork = 'i{0}'.format(work)
        if strwork not in dictionary:
            dictionary[strwork]=cnt
            cnt+=1
    corpus.append([(dictionary['i{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].work_id, itm[1].rating)])
invdict = dict(zip(dictionary.values(), dictionary.keys()))

In [14]:
from gensim.models.lsimodel import LsiModel

lsimodel = LsiModel(corpus, num_topics=num_topic)

In [15]:
record_positive = record.groupby(by='user_id').agg(lambda x: list(x))

corpus = []
for itm in record_positive.iterrows():
    corpus.append([(dictionary['i{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].work_id, itm[1].rating) if 'i{0}'.format(i) in dictionary])

user_lsi_positive = []
for i, topic in enumerate(lsimodel[corpus]):
    temp = np.zeros(num_topic)
    for itm in topic:
        temp[itm[0]] = itm[1]
    user_lsi_positive.append(temp)
user_lsi_positive = np.vstack(user_lsi_positive)
u6 = pd.DataFrame(data = user_lsi_positive, index=record_positive.index, columns=['user_lsi_{0}'.format(i) for i in range(num_topic)])

u6.reset_index().to_csv(data+"features/user_lsi_{0}d.csv".format(num_topic), index=False)

In [7]:
u6.head()

,user_lsi_0,user_lsi_1,user_lsi_2,user_lsi_3,user_lsi_4,user_lsi_5,user_lsi_6,user_lsi_7,user_lsi_8,user_lsi_9
user_id,,,,,,,,,,
0,1.550102,0.184727,1.420607,-1.484637,-0.104344,-0.957708,0.300948,0.333724,-0.631171,-0.278322
1,3.175376,-0.852548,0.193455,-0.825350,-0.168458,-0.035739,0.353309,-0.141072,1.551080,0.187146
2,2.027113,-0.800671,0.495223,-0.685109,-0.011980,0.846306,0.165598,-0.043316,0.598268,0.641669
3,7.769884,-7.004141,1.018471,0.427918,-1.493721,0.710533,3.962426,-0.102615,2.698168,0.787107
4,5.456074,3.889428,-0.441991,-2.041339,-0.492546,0.685152,-0.064113,-0.539779,-0.639626,-0.064427


In [22]:
record_positive = record
record_positive_effectiveitem = record_positive[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
record_positive_effectiveitem = record_positive_effectiveitem[record_positive_effectiveitem.user_count>5]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='work_id').agg(lambda x: list(x))
record_positive = pd.concat((record_positive, record_positive_effectiveitem), axis=1, join='inner')

corpus = []
dictionary = {}
cnt=0
ratlut = {'love': 1.5, 'like': 1, 'neutral': 0.5, 'dislike':-1}
for itm in record_positive.iterrows():
    for user in itm[1].user_id:
        struser = 'u{0}'.format(user)
        if struser not in dictionary:
            dictionary[struser]=cnt
            cnt+=1
    corpus.append([(dictionary['u{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].user_id, itm[1].rating)])
invdict = dict(zip(dictionary.values(), dictionary.keys()))

lsimodel = LsiModel(corpus, num_topics=num_topic)

record_positive = record.groupby(by='work_id').agg(lambda x: list(x))

corpus = []
for itm in record_positive.iterrows():
    corpus.append([(dictionary['u{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].user_id, itm[1].rating) if 'u{0}'.format(i) in dictionary])

item_lsi_positive = []
for i, topic in enumerate(lsimodel[corpus]):
    temp = np.zeros(num_topic)
    for itm in topic:
        temp[itm[0]] = itm[1]
    item_lsi_positive.append(temp)
item_lsi_positive = np.vstack(item_lsi_positive)
i6 = pd.DataFrame(data = item_lsi_positive, index=record_positive.index, columns=['work_lsi_{0}'.format(i) for i in range(num_topic)])

i6.reset_index().to_csv(data+"features/item_lsi_{0}d.csv".format(num_topic), index=False)

## Training set feature (need cv processing)

lsi

In [3]:
train_full.head()

,user_id,work_id,rating,cv
0,50,4041,0,3
1,508,1713,0,1
2,1780,7053,1,2
3,658,8853,0,2
4,1003,9401,0,3


In [2]:
dim = 20

In [5]:
train_user = train_full[['user_id', 'work_id', 'rating']].groupby(by='user_id').agg(lambda x: list(x))

In [6]:
train_user.head()

,work_id,rating
user_id,,
1,"[3371, 9401, 7677, 2799]","[0, 0, 1, 1]"
2,"[8359, 9401, 5700, 4633, 1836, 7430, 3398]","[0, 0, 0, 0, 0, 0, 0]"
3,"[916, 5460, 5434, 2699, 9812]","[0, 1, 1, 1, 0]"
4,"[9692, 5616]","[0, 1]"
5,"[9493, 4041, 2220, 1346, 347]","[0, 0, 1, 0, 0]"


In [11]:
from gensim.models.lsimodel import LsiModel

t = train_full
train_effectiveuser = t[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
train_effectiveuser = train_effectiveuser[train_effectiveuser.work_count>3]
t = t[['user_id', 'work_id', 'rating']].groupby(by='user_id').agg(lambda x: list(x))
t = pd.concat((t, train_effectiveuser), axis=1, join='inner')

corpus = []
dictionary = {}
cnt=0

for itm in t.iterrows():
    for work in itm[1].work_id:
        strwork = 'i{0}'.format(work)
        if strwork not in dictionary:
            dictionary[strwork]=cnt
            cnt+=1
    corpus.append([(dictionary['i{0}'.format(i)], 2*r-1) for i,r in zip(itm[1].work_id, itm[1].rating)])
invdict = dict(zip(dictionary.values(), dictionary.keys()))

lsimodel = LsiModel(corpus, num_topics=dim)

t = train_full[['user_id', 'work_id', 'rating']].groupby(by='user_id').agg(lambda x: list(x))

corpus = []
for itm in t.iterrows():
    corpus.append([(dictionary['i{0}'.format(i)], 2*r-1) for i,r in zip(itm[1].work_id, itm[1].rating) if 'i{0}'.format(i) in dictionary])

user_lsi = []
for i, topic in enumerate(lsimodel[corpus]):
    temp = np.zeros(dim)
    for itm in topic:
        temp[itm[0]] = itm[1]
    user_lsi.append(temp)
user_lsi = np.vstack(user_lsi)
u7 = pd.DataFrame(data = user_lsi, index=t.index, columns=['userwish_lsi_{0}'.format(i) for i in range(dim)])

u7.reset_index().to_csv(data+"features/userwish_lsi_{0}d.csv".format(dim), index=False)

In [14]:
for cv in [1,2,3]:
    train = train_full[~train_full.cv.isin([cv])]
    
    t = train
    train_effectiveuser = t[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
    train_effectiveuser = train_effectiveuser[train_effectiveuser.work_count>3]
    t = t[['user_id', 'work_id', 'rating']].groupby(by='user_id').agg(lambda x: list(x))
    t = pd.concat((t, train_effectiveuser), axis=1, join='inner')

    corpus = []
    dictionary = {}
    cnt=0

    for itm in t.iterrows():
        for work in itm[1].work_id:
            strwork = 'i{0}'.format(work)
            if strwork not in dictionary:
                dictionary[strwork]=cnt
                cnt+=1
        corpus.append([(dictionary['i{0}'.format(i)], 2*r-1) for i,r in zip(itm[1].work_id, itm[1].rating)])
    invdict = dict(zip(dictionary.values(), dictionary.keys()))

    lsimodel = LsiModel(corpus, num_topics=dim)

    t = train[['user_id', 'work_id', 'rating']].groupby(by='user_id').agg(lambda x: list(x))

    corpus = []
    for itm in t.iterrows():
        corpus.append([(dictionary['i{0}'.format(i)], 2*r-1) for i,r in zip(itm[1].work_id, itm[1].rating) if 'i{0}'.format(i) in dictionary])

    user_lsi = []
    for i, topic in enumerate(lsimodel[corpus]):
        temp = np.zeros(dim)
        for itm in topic:
            temp[itm[0]] = itm[1]
        user_lsi.append(temp)
    user_lsi = np.vstack(user_lsi)
    u7 = pd.DataFrame(data = user_lsi, index=t.index, columns=['userwish_lsi_{0}'.format(i) for i in range(dim)])

    u7.reset_index().to_csv(data+"features/{1}/userwish_lsi_{0}d.csv".format(dim, cv), index=False)

In [17]:
t = train_full
train_effectivework = t[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
train_effectivework = train_effectivework[train_effectivework.user_count>3]
t = t[['user_id', 'work_id', 'rating']].groupby(by='work_id').agg(lambda x: list(x))
t = pd.concat((t, train_effectivework), axis=1, join='inner')

corpus = []
dictionary = {}
cnt=0

for itm in t.iterrows():
    for user in itm[1].user_id:
        struser = 'u{0}'.format(user)
        if struser not in dictionary:
            dictionary[struser]=cnt
            cnt+=1
    corpus.append([(dictionary['u{0}'.format(i)], 2*r-1) for i,r in zip(itm[1].user_id, itm[1].rating)])
invdict = dict(zip(dictionary.values(), dictionary.keys()))

lsimodel = LsiModel(corpus, num_topics=dim)

t = train_full[['user_id', 'work_id', 'rating']].groupby(by='work_id').agg(lambda x: list(x))

corpus = []
for itm in t.iterrows():
    corpus.append([(dictionary['u{0}'.format(i)], 2*r-1) for i,r in zip(itm[1].user_id, itm[1].rating) if 'u{0}'.format(i) in dictionary])

work_lsi = []
for i, topic in enumerate(lsimodel[corpus]):
    temp = np.zeros(dim)
    for itm in topic:
        temp[itm[0]] = itm[1]
    work_lsi.append(temp)
work_lsi = np.vstack(work_lsi)
i7 = pd.DataFrame(data = work_lsi, index=t.index, columns=['itemwish_lsi_{0}'.format(i) for i in range(dim)])

i7.reset_index().to_csv(data+"features/itemwish_lsi_{0}d.csv".format(dim), index=False)

In [ ]:
for cv in [1,2,3]:
    train = train_full[~train_full.cv.isin([cv])]
    
    t = train
    train_effectivework = t[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
    train_effectivework = train_effectivework[train_effectivework.user_count>3]
    t = t[['user_id', 'work_id', 'rating']].groupby(by='work_id').agg(lambda x: list(x))
    t = pd.concat((t, train_effectivework), axis=1, join='inner')

    corpus = []
    dictionary = {}
    cnt=0

    for itm in t.iterrows():
        for user in itm[1].user_id:
            struser = 'u{0}'.format(user)
            if struser not in dictionary:
                dictionary[struser]=cnt
                cnt+=1
        corpus.append([(dictionary['u{0}'.format(i)], 2*r-1) for i,r in zip(itm[1].user_id, itm[1].rating)])
    invdict = dict(zip(dictionary.values(), dictionary.keys()))

    lsimodel = LsiModel(corpus, num_topics=dim)

    t = train[['user_id', 'work_id', 'rating']].groupby(by='work_id').agg(lambda x: list(x))

    corpus = []
    for itm in t.iterrows():
        corpus.append([(dictionary['u{0}'.format(i)], 2*r-1) for i,r in zip(itm[1].user_id, itm[1].rating) if 'u{0}'.format(i) in dictionary])

    work_lsi = []
    for i, topic in enumerate(lsimodel[corpus]):
        temp = np.zeros(dim)
        for itm in topic:
            temp[itm[0]] = itm[1]
        work_lsi.append(temp)
    work_lsi = np.vstack(work_lsi)
    i7 = pd.DataFrame(data = work_lsi, index=t.index, columns=['itemwish_lsi_{0}'.format(i) for i in range(dim)])

    i7.reset_index().to_csv(data+"features/{1}/itemwish_lsi_{0}d.csv".format(dim, cv), index=False)

w2v feature

In [41]:
w2vdim = 20

In [37]:
from gensim.models.word2vec import Word2Vec
from random import sample

t_pos = train_full[train_full.rating==1]
train_wish = t_pos[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
train_wish = train_wish[train_wish.work_count>3]
t_pos = t_pos[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})
t_pos = pd.concat((t_pos, train_wish), axis=1, join='inner')

sentences = []
for itm in t_pos.iterrows():
    t = ['i{0}'.format(str(i)) for i in itm[1].work_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_positive = Word2Vec(sentences, size=w2vdim)

itemwish_w2v_positive = []
item_index = []
for item in range(9897):
    if 'i{0}'.format(item) in w2v_positive:
        item_index.append(item)
        itemwish_w2v_positive.append(w2v_positive['i{0}'.format(item)])
itemwish_w2v_positive = np.vstack(itemwish_w2v_positive)
i4 = pd.DataFrame(data = itemwish_w2v_positive, index=item_index, columns=['itemwish_w2v_pos_{0}'.format(i) for i in range(w2vdim)])
i4.index.name='work_id'

i4.reset_index().to_csv(data+"features/itemwish_w2v_shuffled_{0}d.csv".format(w2vdim), index=False)

In [38]:
for cv in [1,2,3]:
    train = train_full[~train_full.cv.isin([cv])]
    
    t_pos = train[train.rating==1]
    train_wish = t_pos[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
    train_wish = train_wish[train_wish.work_count>3]
    t_pos = t_pos[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})
    t_pos = pd.concat((t_pos, train_wish), axis=1, join='inner')

    sentences = []
    for itm in t_pos.iterrows():
        t = ['i{0}'.format(str(i)) for i in itm[1].work_list]
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))

    w2v_positive = Word2Vec(sentences, size=w2vdim)

    itemwish_w2v_positive = []
    item_index = []
    for item in range(9897):
        if 'i{0}'.format(item) in w2v_positive:
            item_index.append(item)
            itemwish_w2v_positive.append(w2v_positive['i{0}'.format(item)])
    itemwish_w2v_positive = np.vstack(itemwish_w2v_positive)
    i4 = pd.DataFrame(data = itemwish_w2v_positive, index=item_index, columns=['itemwish_w2v_pos_{0}'.format(i) for i in range(w2vdim)])
    i4.index.name='work_id'

    i4.reset_index().to_csv(data+"features/{1}/itemwish_w2v_shuffled_{0}d.csv".format(w2vdim, cv), index=False)

In [39]:
t_pos = train_full[train_full.rating==1]
train_wish = t_pos[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
train_wish = train_wish[train_wish.user_count>3]
t_pos = t_pos[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})
t_pos = pd.concat((t_pos, train_wish), axis=1, join='inner')

sentences = []
for itm in t_pos.iterrows():
    t = ['u{0}'.format(str(i)) for i in itm[1].user_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_positive = Word2Vec(sentences, size=w2vdim)

userwish_w2v_positive = []
user_index = []
for user in range(1983):
    if 'u{0}'.format(user) in w2v_positive:
        user_index.append(user)
        userwish_w2v_positive.append(w2v_positive['u{0}'.format(user)])
userwish_w2v_positive = np.vstack(userwish_w2v_positive)
u4 = pd.DataFrame(data = userwish_w2v_positive, index=user_index, columns=['userwish_w2v_pos_{0}'.format(i) for i in range(w2vdim)])
u4.index.name='user_id'

u4.reset_index().to_csv(data+"features/userwish_w2v_shuffled_{0}d.csv".format(w2vdim), index=False)

In [40]:
for cv in [1,2,3]:
    train = train_full[~train_full.cv.isin([cv])]
    
    t_pos = train[train.rating==1]
    train_wish = t_pos[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
    train_wish = train_wish[train_wish.user_count>3]
    t_pos = t_pos[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})
    t_pos = pd.concat((t_pos, train_wish), axis=1, join='inner')

    sentences = []
    for itm in t_pos.iterrows():
        t = ['u{0}'.format(str(i)) for i in itm[1].user_list]
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))

    w2v_positive = Word2Vec(sentences, size=w2vdim)

    userwish_w2v_positive = []
    user_index = []
    for user in range(1983):
        if 'u{0}'.format(user) in w2v_positive:
            user_index.append(user)
            userwish_w2v_positive.append(w2v_positive['u{0}'.format(user)])
    userwish_w2v_positive = np.vstack(userwish_w2v_positive)
    u4 = pd.DataFrame(data = userwish_w2v_positive, index=user_index, columns=['userwish_w2v_pos_{0}'.format(i) for i in range(w2vdim)])
    u4.index.name='user_id'

    u4.reset_index().to_csv(data+"features/{1}/userwish_w2v_shuffled_{0}d.csv".format(w2vdim,cv), index=False)

In [42]:
t_pos = train_full[train_full.rating==0]
train_wish = t_pos[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
train_wish = train_wish[train_wish.work_count>3]
t_pos = t_pos[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})
t_pos = pd.concat((t_pos, train_wish), axis=1, join='inner')

sentences = []
for itm in t_pos.iterrows():
    t = ['i{0}'.format(str(i)) for i in itm[1].work_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_positive = Word2Vec(sentences, size=w2vdim)

itemwish_w2v_positive = []
item_index = []
for item in range(9897):
    if 'i{0}'.format(item) in w2v_positive:
        item_index.append(item)
        itemwish_w2v_positive.append(w2v_positive['i{0}'.format(item)])
itemwish_w2v_positive = np.vstack(itemwish_w2v_positive)
i4 = pd.DataFrame(data = itemwish_w2v_positive, index=item_index, columns=['itemwish_w2v_neg_{0}'.format(i) for i in range(w2vdim)])
i4.index.name='work_id'

i4.reset_index().to_csv(data+"features/itemwish_w2vneg_shuffled_{0}d.csv".format(w2vdim), index=False)

for cv in [1,2,3]:
    train = train_full[~train_full.cv.isin([cv])]
    
    t_pos = train[train.rating==0]
    train_wish = t_pos[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
    train_wish = train_wish[train_wish.work_count>3]
    t_pos = t_pos[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})
    t_pos = pd.concat((t_pos, train_wish), axis=1, join='inner')

    sentences = []
    for itm in t_pos.iterrows():
        t = ['i{0}'.format(str(i)) for i in itm[1].work_list]
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))

    w2v_positive = Word2Vec(sentences, size=w2vdim)

    itemwish_w2v_positive = []
    item_index = []
    for item in range(9897):
        if 'i{0}'.format(item) in w2v_positive:
            item_index.append(item)
            itemwish_w2v_positive.append(w2v_positive['i{0}'.format(item)])
    itemwish_w2v_positive = np.vstack(itemwish_w2v_positive)
    i4 = pd.DataFrame(data = itemwish_w2v_positive, index=item_index, columns=['itemwish_w2v_neg_{0}'.format(i) for i in range(w2vdim)])
    i4.index.name='work_id'

    i4.reset_index().to_csv(data+"features/{1}/itemwish_w2vneg_shuffled_{0}d.csv".format(w2vdim, cv), index=False)
    
t_pos = train_full[train_full.rating==0]
train_wish = t_pos[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
train_wish = train_wish[train_wish.user_count>3]
t_pos = t_pos[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})
t_pos = pd.concat((t_pos, train_wish), axis=1, join='inner')

sentences = []
for itm in t_pos.iterrows():
    t = ['u{0}'.format(str(i)) for i in itm[1].user_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_positive = Word2Vec(sentences, size=w2vdim)

userwish_w2v_positive = []
user_index = []
for user in range(1983):
    if 'u{0}'.format(user) in w2v_positive:
        user_index.append(user)
        userwish_w2v_positive.append(w2v_positive['u{0}'.format(user)])
userwish_w2v_positive = np.vstack(userwish_w2v_positive)
u4 = pd.DataFrame(data = userwish_w2v_positive, index=user_index, columns=['userwish_w2v_neg_{0}'.format(i) for i in range(w2vdim)])
u4.index.name='user_id'

u4.reset_index().to_csv(data+"features/userwish_w2vneg_shuffled_{0}d.csv".format(w2vdim), index=False)

for cv in [1,2,3]:
    train = train_full[~train_full.cv.isin([cv])]
    
    t_pos = train[train.rating==0]
    train_wish = t_pos[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
    train_wish = train_wish[train_wish.user_count>3]
    t_pos = t_pos[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})
    t_pos = pd.concat((t_pos, train_wish), axis=1, join='inner')

    sentences = []
    for itm in t_pos.iterrows():
        t = ['u{0}'.format(str(i)) for i in itm[1].user_list]
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))
        sentences.append(sample(t, len(t)))

    w2v_positive = Word2Vec(sentences, size=w2vdim)

    userwish_w2v_positive = []
    user_index = []
    for user in range(1983):
        if 'u{0}'.format(user) in w2v_positive:
            user_index.append(user)
            userwish_w2v_positive.append(w2v_positive['u{0}'.format(user)])
    userwish_w2v_positive = np.vstack(userwish_w2v_positive)
    u4 = pd.DataFrame(data = userwish_w2v_positive, index=user_index, columns=['userwish_w2v_neg_{0}'.format(i) for i in range(w2vdim)])
    u4.index.name='user_id'

    u4.reset_index().to_csv(data+"features/{1}/userwish_w2vneg_shuffled_{0}d.csv".format(w2vdim,cv), index=False)